In [1]:
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import gmaps
# Import API key
from api_keys import geoapify_key

In [2]:
csv_file = Path(r"output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,celano,42.0855,13.5398,16.45,61,1,0.45,IT,1715496405
1,1,atwater,37.3477,-120.6091,22.86,63,0,1.54,US,1715496407
2,2,camacupa,-12.0167,17.4833,18.01,53,9,1.69,AO,1715496407
3,3,grytviken,-54.2811,-36.5092,1.38,89,97,3.66,GS,1715496408
4,4,berezovyy,51.6667,135.7000,4.12,86,100,3.25,RU,1715496409


In [3]:
import warnings
warnings.filterwarnings("ignore")

# Configure the map plot
city_data_df.hvplot.points("Lng","Lat", geo = True, tiles="OSM", \
                                        frame_width=700, frame_height=500, \
                                        size="Humidity", scale=0.5, color="City")


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
temp_condition = ((city_data_df["Max Temp"] > 20) & (city_data_df["Max Temp"] < 27))
wind_condition = (city_data_df["Wind Speed"] < 5)
#humid_condition = ((city_data_df["Humidity"] > 20) & (city_data_df["Humidity"] < 60))
#note - removing the above condition, since its resulting in a fewer data set.
cloud_condition = (city_data_df["Cloudiness"] < 10)

city_data_fil = city_data_df.loc[(temp_condition & wind_condition & cloud_condition ), :]

# Drop any rows with null values
city_final = city_data_fil.dropna()

# Display sample data
city_final

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,atwater,37.3477,-120.6091,22.86,63,0,1.54,US,1715496407
26,26,adamstown,-25.0660,-130.1015,20.36,65,0,3.66,PN,1715496424
39,39,varadero,23.1536,-81.2514,23.42,82,0,1.69,CU,1715496432
53,53,la romana,18.4273,-68.9728,25.22,86,0,2.78,DO,1715496440
56,56,sarina,-21.4333,149.2167,25.62,75,0,2.72,AU,1715496442
84,84,trinidad,-14.8333,-64.9000,23.28,77,0,1.87,BO,1715496459
131,131,kas,36.2018,29.6377,20.37,74,1,2.74,TR,1715496494
150,150,arraial do cabo,-22.9661,-42.0278,23.63,86,0,3.26,BR,1715496512
164,164,khost,33.3395,69.9204,25.99,29,0,3.82,AF,1715496520
172,172,cockburn town,21.4612,-71.1419,25.99,84,0,2.47,TC,1715496524


In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_final.copy()

hotel_df = hotel_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,atwater,US,37.3477,-120.6091,63,
26,adamstown,PN,-25.0660,-130.1015,65,
39,varadero,CU,23.1536,-81.2514,82,
53,la romana,DO,18.4273,-68.9728,86,
56,sarina,AU,-21.4333,149.2167,75,
84,trinidad,BO,-14.8333,-64.9000,77,
131,kas,TR,36.2018,29.6377,74,
150,arraial do cabo,BR,-22.9661,-42.0278,86,
164,khost,AF,33.3395,69.9204,29,
172,cockburn town,TC,21.4612,-71.1419,84,


In [6]:
radius = 10000
limit = 5
apiKey = geoapify_key
params = {
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index, "Lng"]
    latitude = hotel_df.loc[index, "Lat"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    params["categories"] = f'accommodation.hotel'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
atwater - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
varadero - nearest hotel: Hotel Cuatro Palmas
la romana - nearest hotel: Hotel Frano
sarina - nearest hotel: Tandara Hotel
trinidad - nearest hotel: Hotel Palace
kas - nearest hotel: KEKOVA OTEL
arraial do cabo - nearest hotel: No hotel found
khost - nearest hotel: No hotel found
cockburn town - nearest hotel: The Salt Raker Inn
newman - nearest hotel: No hotel found
paraty - nearest hotel: Solar dos Gerânios
bonab - nearest hotel: هتل لاله بناب
nova vicosa - nearest hotel: Pousada Pontal da Barra
yeppoon - nearest hotel: Blossom's on Seaspray
tafresh - nearest hotel: هتل پردیسان
andergrove - nearest hotel: No hotel found
hood river - nearest hotel: Hood River Hotel
sao joao da barra - nearest hotel: Pousada Mediterrâneo
bechar - nearest hotel: Hôtel Madania
namibe - nearest hotel: Nelsal Pensao
palmas - nearest hotel: Hotel Triângulo Mineiro
quartu sant'elena - nearest hotel: Hotel 

,City,Country,Lat,Lng,Humidity,Hotel Name
1,atwater,US,37.3477,-120.6091,63,No hotel found
26,adamstown,PN,-25.0660,-130.1015,65,No hotel found
39,varadero,CU,23.1536,-81.2514,82,Hotel Cuatro Palmas
53,la romana,DO,18.4273,-68.9728,86,Hotel Frano
56,sarina,AU,-21.4333,149.2167,75,Tandara Hotel
84,trinidad,BO,-14.8333,-64.9000,77,Hotel Palace
131,kas,TR,36.2018,29.6377,74,KEKOVA OTEL
150,arraial do cabo,BR,-22.9661,-42.0278,86,No hotel found
164,khost,AF,33.3395,69.9204,29,No hotel found
172,cockburn town,TC,21.4612,-71.1419,84,The Salt Raker Inn


In [7]:
%%capture --no-display


import warnings
warnings.filterwarnings("ignore")

# Configure the map plot
city_data_df.hvplot.points("Lng","Lat", geo = True, tiles="OSM", \
                                        frame_width=700, frame_height=500, \
                                        size="Humidity", scale=0.5, color="City")

# Configure the map plot
hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)

# Display the map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)

In [8]:
print("end file")

end file
